# Exploration simulation of a Molten NaCl Salt with OpenMM

This notebook performs an NpT equilibration followed by an NVE production run.
The NVE simulation is intended for a first analysis of a system,
to get a basic idea of the time scales of this system,
which will be used to determine the simulation time and block size for the production run.

The Born-Huggins-Mayer-Tosi-Fumi force field for NaCl is implemented in `bhmtf.py`.
The MD implementation and some related utility functions can be found in `utils.py`.
This notebook merely ties these two together and performs some basic sanity checks on the reults.

You can use this notebook as such, but it is also designed to be used as a step in a StepUp workflow
that performs all the MD simulations for a reasonably accurate computation of the conductivity of NaCl.

In [ ]:
import mdtraj
import nglview
from bhmtf import add_nacl_forces, build_nacl_lattice
from openmm import unit
from openmm.app import PDBFile
from stepup.core.api import amend
from utils import make_plots, runmd

In [ ]:
# Inform StepUp of the output files it should expect, and which files are used as inputs.
amend(
    inp=[
        "bhmtf.py",
        "utils.py",
    ],
    out=[
        "output/exploration_init.pdb",
        "output/exploration_npt_final.pdb",
        "output/exploration_npt_scalars.csv",
        "output/exploration_npt_traj.dcd",
        "output/exploration_npt_traj.npz",
        "output/exploration_nve_final.pdb",
        "output/exploration_nve_scalars.csv",
        "output/exploration_nve_traj.dcd",
        "output/exploration_nve_traj.npz",
    ],
)

## Simulation settings and initial state

In [ ]:
# Physical parameters
temperature = 1100 * unit.kelvin
pressure = 1 * unit.bar

# The simulation density from https://doi.org/10.1021/jp5050332 is only used for initialization.
density = 1.4444 * unit.gram / unit.centimeter**3

# Time-related settings
timestep = 5 * unit.femtosecond
stride = 10
nstep_npt = 10000
tau_thermostat = 1 * unit.picosecond
nstep_nve = 10000

In [ ]:
# Build initial state (1728 = (6*2)**3 ions)
# Same settings as in Wang 2020 (https://doi.org/10.1063/5.0023225)
system, topology, atnums, atcoords_init = build_nacl_lattice(6, density)
add_nacl_forces(system, topology, do_charge=True, cutoff=1.5 * unit.nanometer)

# Visualize the initial geometry.
with open("output/exploration_init.pdb", "w") as f:
    PDBFile.writeFile(topology, atcoords_init, f)
view = nglview.show_mdtraj(mdtraj.load("output/exploration_init.pdb"))
view.add_unitcell()
view

## Equilibration in the NPT ensemble

In [ ]:
atcoords_npt, atvels_npt = runmd(
    "exploration_npt",
    system,
    topology,
    atcoords_init,
    nstep=nstep_npt,
    timestep=timestep,
    stride=stride,
    temperature=temperature,
    pressure=pressure,
)

In [ ]:
traj = mdtraj.load("output/exploration_npt_traj.dcd", top="output/exploration_init.pdb")
view = nglview.show_mdtraj(traj)
view.add_unitcell()
view

In [ ]:
make_plots("exploration_npt")

## Production in the NVE ensemble

In [ ]:
atcoords_end = runmd(
    "exploration_nve",
    system,
    topology,
    atcoords_npt,
    nstep=nstep_nve,
    timestep=timestep,
    stride=stride,
    atvels=atvels_npt,
)

In [ ]:
traj = mdtraj.load("output/exploration_nve_traj.dcd", top="output/exploration_init.pdb")
view = nglview.show_mdtraj(traj)
view.add_unitcell()
view

In [ ]:
make_plots("exploration_nve")